In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [ ]:
data_path='/kaggle/input/blood-cells/dataset2-master/dataset2-master/images'

In [ ]:
TRAIN_PATH = data_path+'/'+'TRAIN'
TEST_PATH = data_path +'/'+'TEST'
VALID_PATH = data_path+'/'+'TEST_SIMPLE'

In [ ]:
import cv2 
import matplotlib.pyplot as plt
from keras.preprocessing.image import *
from keras.applications import InceptionV3,VGG16
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers
from keras.models import *

In [ ]:
sample_data1_path = os.path.join(TRAIN_PATH+'/MONOCYTE','_11_3865.jpeg')
sample_data2_path = os.path.join(TRAIN_PATH+'/EOSINOPHIL','_41_6558.jpeg')
sample_data3_path = os.path.join(TRAIN_PATH+'/LYMPHOCYTE','_14_8262.jpeg')
sample_data4_path = os.path.join(TRAIN_PATH+'/NEUTROPHIL','_28_8416.jpeg')
sample_data1 = cv2.imread(sample_data1_path)
sample_data2 = cv2.imread(sample_data2_path)
sample_data3 = cv2.imread(sample_data3_path)
sample_data4 = cv2.imread(sample_data4_path)
img_shape =sample_data3.shape 
fig = plt.figure(figsize=(9,9))
ax1 = fig.add_subplot(2, 2, 1,xticks=[],yticks=[],title='MONOCYTE')
ax1.imshow(sample_data1)
ax2 = fig.add_subplot(2, 2, 2,xticks=[],yticks=[],title='EOSINOPHIL')
ax2.imshow(sample_data2)
ax3 = fig.add_subplot(2, 2, 3,xticks=[],yticks=[],title='LYMPHOCYTE')
ax3.imshow(sample_data3)
ax4 = fig.add_subplot(2, 2, 4,xticks=[],yticks=[],title='NEUTROPHIL')
ax4.imshow(sample_data4)

In [ ]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

Already, The images are aumentated, So i didn't anything.

In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(img_shape[0],img_shape[1]),
    batch_size=32,
    class_mode='categorical',
    seed=2019
    ,color_mode='rgb'  
)
valid_generator = valid_datagen.flow_from_directory(
    VALID_PATH,
    target_size=(img_shape[0],img_shape[1]),
    batch_size=32,
    class_mode='categorical',
    seed=2019
    ,color_mode='rgb'  
)
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(img_shape[0],img_shape[1]),
    batch_size=32,
    shuffle=False,
    class_mode='categorical',
    seed=2019
    ,color_mode='rgb'
)

In [ ]:
def get_model():
    base_mdoel = InceptionV3(weights='imagenet',include_top=False,input_shape=img_shape)
    model= Sequential()
    model.add(base_mdoel)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))
    model.summary()
    optimizer = optimizers.Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model

In [ ]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

In [ ]:
batch_size=32
model_path = '../model/'
if not os.path.exists(model_path):
    os.mkdir(model_path)
    
model_path = model_path + 'best_model.hdf5'
train_num = len(os.listdir(TRAIN_PATH+'/MONOCYTE')) +len(os.listdir(TRAIN_PATH+'/EOSINOPHIL')) +len(os.listdir(TRAIN_PATH+'/LYMPHOCYTE')) +len(os.listdir(TRAIN_PATH+'/NEUTROPHIL'))
test_num = len(os.listdir(VALID_PATH+'/MONOCYTE')) +len(os.listdir(TEST_PATH+'/EOSINOPHIL')) +len(os.listdir(TEST_PATH+'/LYMPHOCYTE')) +len(os.listdir(TEST_PATH+'/NEUTROPHIL'))
valid_num = len(os.listdir(VALID_PATH+'/MONOCYTE')) +len(os.listdir(VALID_PATH+'/EOSINOPHIL')) +len(os.listdir(VALID_PATH+'/LYMPHOCYTE')) +len(os.listdir(VALID_PATH+'/NEUTROPHIL'))

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',patience=2,mode='min',verbose=1),
            ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 1, min_lr=0.00001, verbose=1, mode='min'),
            ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min'),]

In [ ]:
my_inception_model =get_model()


history = my_inception_model.fit_generator(
    train_generator,
    steps_per_epoch=get_steps(entire_num, batch_size),
    epochs=100,
    validation_data=valid_generator,
    validation_steps=get_steps(valid_num, batch_size),
    verbose=1,
    callbacks = callbacks
)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'bo', label='Traing loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Trainging and validation loss')
plt.legend()
plt.show()


In [ ]:
test_generator.reset()
my_inception_model.load_weights(model_path)
scores = my_inception_model.evaluate_generator(test_generator,steps=get_steps(test_num,batch_size),verbose=1)

In [ ]:
print('test loss =',scores[0],", test acc :",scores[1])